In [ ]:
from tqdm import tqdm_notebook
import pathlib
import os
import numpy as np
import nbimporter
# Import our data_modification.ipynb as Modify to use functions from it
import data_modification_inf as Modify

In [ ]:
# Create the Folder for the Split Data and Get the PATHS of the Split Folders and Modified Folders
# FOLDER_PATHS, SPLIT_FOLDER_PATHS = Modify.loadcase_folder_paths(MODIFIED_DATA_PATH,SPLIT_DATA_PATH)
# Pass the Modified Folder Paths and get the Names and Paths of the different Data instances
# modified_data_names, modified_data_paths = Modify.get_data_names_paths(FOLDER_PATHS)
# So FOLDER_PATHS =  PATHS of the MODIFIED FOLDERS, SPLIT_FOLDER_PATHS = PATHS of SPLIT Folders
# MODIFIED_DATA_NAMES and _PATHS = all names and the PATHS of the modified data
# All Data is split into n with n being the Amount of Loadcase Subfolders, in this Case 1
# print(modified_data_names)




In [ ]:
# Add a border for the Dataset, when we have a 64x48 Dimension
# Question is: will it work without problems?
def add_padding(array,x_offset=1,y_offset=0):
    x_shape_old = np.shape(array)[0]
    y_shape_old = np.shape(array)[1]
    x_new = x_shape_old+1
    y_new = y_shape_old+1
    new = np.zeros((x_new,y_new))
    new[x_offset:array.shape[0]+x_offset,y_offset:array.shape[1]+y_offset] = array
    return new


In [ ]:
# Iteration Skip is the Amount of Iterations skipped from the 0-th Iteration in the Output
def split_data(MODIFIED_FOLDER_PATH, MODIFIED_DATA_NAMES,SPLIT_FOLDER_PATH,ITERATION_SKIP,padding = True):
    UNDER_ITERATION_SKIP = []
    # Check for the Checkpoint and Faulty data
    if all(x in MODIFIED_DATA_NAMES for x in ['checkpoint.npy', 'faulty.npy']):
        MODIFIED_DATA_NAMES = np.array(MODIFIED_DATA_NAMES[2:])
        print ("Checkpoint and Faulty found, skipping them, starting at: ", MODIFIED_DATA_NAMES[0])

    for i,s in tqdm_notebook(enumerate(MODIFIED_DATA_NAMES)):
# Loads the Modified Data
        modified_data = np.load(os.path.join(MODIFIED_FOLDER_PATH,s),allow_pickle=True)
        input_folder_path =os.path.join(SPLIT_FOLDER_PATH, 'Input')
        os.makedirs(input_folder_path, exist_ok= True)
#         print(get_shape)
# Creates The Path and name for the split data Input and Output
        split_path_input= os.path.join(input_folder_path,s)

        if padding == True:
            modified_data[0] = add_padding(modified_data[0])
            modified_data[3] = add_padding(modified_data[3])
            modified_data[4] = add_padding(modified_data[4])
            modified_data[5] = add_padding(modified_data[5])
            np.save(split_path_input,[np.array(modified_data[0]),np.array(modified_data[1]),
                                      np.array(modified_data[2]),np.array(modified_data[3]),
                                      np.array(modified_data[4]),np.array(modified_data[5])])
        else:
#             Seperate into Channels 
            np.save(split_path_input,[np.array(modified_data[0]),np.array(modified_data[1]),
                                      np.array(modified_data[2]),np.array(modified_data[3]),
                                      np.array(modified_data[4]),np.array(modified_data[5])])

In [ ]:
def full_auto_split(MODIFIED_DATA_PATH,SPLIT_DATA_PATH,MAX_ITER=300):
    FOLDER_PATHS, SPLIT_FOLDER_PATHS = Modify.loadcase_folder_paths(MODIFIED_DATA_PATH,SPLIT_DATA_PATH)
    modified_data_names, modified_data_paths = Modify.get_data_names_paths(FOLDER_PATHS)
#     print(modified_data_names,modified_data_paths)
    for i,s in enumerate(FOLDER_PATHS):
        os.makedirs(SPLIT_DATA_PATH, exist_ok=True)
        split_data(FOLDER_PATHS[i],modified_data_names[i],SPLIT_FOLDER_PATHS[i],MAX_ITER)
        
full_auto_split(MODIFIED_DATA_PATH,SPLIT_DATA_PATH)